Código para la creación de imagen tiff con metadatos de drone

In [57]:
from PIL import Image
from PIL.TiffTags import TAGS
import functools
import operator
import sys
import re
import rasterio

In [58]:
# obtenemos la imagen .tif
imagen = "./data/fotos/DJI_0051.TIF"
img = Image.open(imagen)
# obtenemos los Tags de la metadata y se almacenan en un diccionario
meta_dict = {TAGS[key]: img.tag[key] for key in img.tag_v2}
# Se imprime el diccionario para obtener la composición de los datos
# for rec in meta_dict:
#     print(rec, ":", meta_dict[rec])


In [59]:
# Extraemos el indicador xmp
p = meta_dict.get("XMP")
s = p.decode("UTF-8")
# dividimos por el salto de linea y obtenemos una lista
div = s.split("\n")
type(div)


list

In [60]:
#eliminamos los espacios vacios 

for ind,recorrido in enumerate(div):
    div[ind]=recorrido.strip() 

div[17]

'drone-dji:AbsoluteAltitude="+56.59"'

In [61]:
usar = div[17]

result = re.search(":(.*)=", div[17])
result.group(1)

result2 = re.search("\"(.*)\"", div[17])
result2.group(1)

print(result.group(1))
print(result2.group(1))


AbsoluteAltitude
+56.59


Creo un diccionario donde almaceno las variables de la metadata con su valor

In [62]:
metadiccionario = {}
for ind,recorrido in enumerate(div):
    try:
        metadiccionario[re.search(":(.*)=", div[ind]).group(1)] = re.search("\"(.*)\"", div[ind]).group(1)
    except:
        pass

print(metadiccionario)

{'xmpmeta xmlns:x': 'adobe:ns:meta/', 'RDF xmlns:rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#', 'Description rdf:about': 'DJI Meta Data', 'tiff': 'http://ns.adobe.com/tiff/1.0/', 'exif': 'http://ns.adobe.com/exif/1.0/', 'xmp': 'http://ns.adobe.com/xap/1.0/', 'xmpMM': 'http://ns.adobe.com/xap/1.0/mm/', 'dc': 'http://purl.org/dc/elements/1.1/', 'crs': 'http://ns.adobe.com/camera-raw-settings/1.0/', 'drone-dji': 'http://www.dji.com/drone-dji/1.0/', 'Camera': 'http://pix4d.com/camera/1.0', 'ModifyDate': '2022-11-29', 'CreateDate': '2022-11-29', 'Make': 'DJI', 'Model': 'FC6360', 'format': 'image/TIF', 'AbsoluteAltitude': '+56.59', 'RelativeAltitude': '+19.80', 'GpsLatitude': '10.55178659', 'GpsLongitude': '-74.22021469', 'GimbalRollDegree': '+0.00', 'GimbalYawDegree': '-84.90', 'GimbalPitchDegree': '-89.90', 'FlightRollDegree': '+5.60', 'FlightYawDegree': '-84.80', 'FlightPitchDegree': '-2.70', 'FlightXSpeed': '-0.30', 'FlightYSpeed': '+0.00', 'FlightZSpeed': '+0.00', 'CamReverse': '0

Se calcula la resolución en metros por pixel.
Es decir, a cuantos metros equivale un pixel

In [63]:
resolucion = float(metadiccionario["AbsoluteAltitude"])/float(metadiccionario["CalibratedFocalLength"])
resolucion

0.029576654423632085

In [64]:
# Abrir la imagen
with rasterio.open(imagen) as src:
    # Obtener la información de la imagen
    width = src.width
    height = src.height
    transform = src.transform
    crs = src.crs


Cálculo de las coordenadas de las esquinas la imagen

Se divide entre 111111 para convertir los metros en grados. Esto se debe a que 1 grado de longitud o latitud en el ecuador es aproximadamente 111.111 km de ancho. Dado que la longitud y la latitud son medidas en grados, se debe convertir el tamaño de los píxeles en metros a grados para poder calcular las coordenadas extremas en grados.
Donde el ancho de la imagen en metros se divide entre la distancia en metros por grado para obtener la diferencia en grados. El factor 111111 es una aproximación, ya que la distancia en metros por grado varía según la latitud. Sin embargo, esta aproximación es suficiente para una precisión razonable.

Es importante tener en cuenta que esta conversión solo es válida para latitudes cercanas al ecuador, ya que la distancia en metros por grado varía según la latitud.

In [65]:
# Calcular las coordenadas de los extremos en metros
min_lon = float(metadiccionario["GpsLongitude"]) - (
    float(metadiccionario["CalibratedOpticalCenterX"]) * resolucion) / 111111
max_lon = float(metadiccionario["GpsLongitude"]) + (
    float(metadiccionario["CalibratedOpticalCenterX"]) * resolucion) / 111111
min_lat = float(metadiccionario["GpsLatitude"]) - (
    float(metadiccionario["CalibratedOpticalCenterY"]) * resolucion) / 111111
max_lat = float(metadiccionario["GpsLatitude"]) + (
    float(metadiccionario["CalibratedOpticalCenterY"]) * resolucion) / 111111
print(min_lon, max_lon, min_lat, max_lat)


-74.2204276421248 -74.22000173787521 10.551613566398599 10.551959613601403


In [66]:
transform = rasterio.transform.from_bounds(
    min_lon, min_lat, max_lon, max_lat, img.width, img.height)
transform


Affine(2.6619015599749216e-07, 0.0, -74.2204276421248,
       0.0, -2.661901560029579e-07, 10.551959613601403)

Leo la imagen con rasterio y luego la guardo con las coordenadas correspondientes y como archivo tiff

In [67]:
# Abrir el archivo original
with rasterio.open(imagen) as src:
    # Leer los datos de la imagen
    image = src.read()

    # Crear una copia del archivo
    with rasterio.open(
        'data/test_drone05.tiff',
        'w',
        driver='GTiff',
        height=src.height,
        width=src.width,
        count=1,
        dtype="float64",
        crs={"init": "epsg:4326"},
        transform=transform,
    ) as dst:
        dst.write(image)
